In [2]:
import pandas as pd
import scipy.stats as stats
import numpy as np
import plotly.express as px
import matplotlib as mpl
from sklearn.model_selection import train_test_split

mpl.rc('font', family='AppeGothic')

# 특성공학 (Pipe Line)

In [5]:
df1 = pd.read_csv('01_Data.csv')

In [6]:
df1

,Index,Member_ID,Sales_Type,Contract_Type,Channel,Datetime,Term,Payment_Type,Product_Type,Amount_Month,Customer_Type,Age,Address1,Address2,State,Overdue_count,Overdue_Type,Gender,Credit_Rank,Bank
0,1,66758234,렌탈,일반계약,영업방판,2022-05-05,60,CMS,DES-1,96900,개인,42.0,경기도,경기도,계약확정,0,없음,여자,9.0,새마을금고
1,2,66755948,렌탈,교체계약,영업방판,2023-02-19,60,카드이체,DES-1,102900,개인,39.0,경기도,경기도,계약확정,0,없음,남자,2.0,현대카드
2,3,66756657,렌탈,일반계약,홈쇼핑/방송,2022-02-27,60,CMS,DES-1,96900,개인,48.0,경기도,경기도,계약확정,0,없음,여자,8.0,우리은행
3,4,66423450,멤버십,멤버십3유형,재계약,2022-05-12,12,CMS,DES-1,66900,개인,39.0,경기도,경기도,계약확정,0,없음,남자,5.0,농협회원조합
4,5,66423204,멤버십,멤버십3유형,재계약,2022-05-09,12,CMS,DES-1,66900,개인,60.0,경기도,경기도,기간만료,12,있음,남자,8.0,농협회원조합
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51296,51298,66579515,렌탈,프로모션계약,대형마트A,2022-02-28,60,CMS,DES-3A,96900,개인,47.0,경기도,경기도,계약확정,0,없음,남자,NaN,기업은행
51297,51299,66799558,렌탈,일반계약,대형마트A,2022-03-31,60,CMS,DES-1,96900,개인,42.0,경기도,경기도,계약확정,0,없음,여자,8.0,새마을금고
51298,51300,66799197,렌탈,프로모션계약,영업방판,2022-03-31,39,카드이체,ERA,120900,개인,65.0,서울특별시,서울특별시,계약확정,0,없음,여자,1.0,롯데카드
51299,51301,66792778,렌탈,일반계약,홈쇼핑/방송,2023-02-05,60,카드이체,DES-1,96900,개인,54.0,서울특별시,서울특별시,계약확정,0,없음,여자,2.0,롯데카드


## Imputation

- 결측값을 다른 값으로 대치 (fillna)

In [7]:
from sklearn.impute import SimpleImputer #단순대치(fillna)

In [8]:
df1['Credit_Rank'].isnull().sum()

8781

In [10]:
df1['CR_clean'] = SimpleImputer(strategy='mean').fit_transform(df1[['Credit_Rank']])

In [11]:
df1[['Credit_Rank','CR_clean']].tail(7)

,Credit_Rank,CR_clean
51294,NaN,3.42881
51295,1.0,1.00000
51296,NaN,3.42881
51297,8.0,8.00000
51298,1.0,1.00000
51299,2.0,2.00000
51300,8.0,8.00000


In [12]:
# 문자데이터
df1['Bank'].iloc[200:210]

200    새마을금고
201     하나은행
202    농협중앙회
203     신한은행
204     신한은행
205      NaN
206      NaN
207      NaN
208      NaN
209    농협중앙회
Name: Bank, dtype: object

In [16]:
# 문자 항목에 대해 최빈값으로 결측값을 처리
s1 = SimpleImputer(strategy = 'most_frequent').fit_transform(df1[['Bank']])
pd.DataFrame(s1).iloc[200:210]

,0
200,새마을금고
201,하나은행
202,농협중앙회
203,신한은행
204,신한은행
205,롯데카드
206,롯데카드
207,롯데카드
208,롯데카드
209,농협중앙회


## Scaling & Encoding

- Scaling : 서로 다른 숫자데이터의 Scale을 조정하여 학습
- Encoding : 문자 데이터를 숫자 형태로 변환하여 학습

In [17]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler

In [19]:
# StandardScaler : 평균이 0, 표준편차 1 형태로 데이터를 변환
# 선형회귀와 같은 전통적 통계방식의 연산을 수행하거나, 선형대수 기반의 학습 알고리즘
# 이상치가 없고, 정규성을 잘 따르는 데이터에 대해 적용
df1[['Amount_Month', 'Term', 'Age']].describe()

,Amount_Month,Term,Age
count,51301.000000,51301.000000,44329.000000
mean,93994.974289,55.639149,50.024093
std,15304.263988,12.009915,10.983877
min,54603.000000,12.000000,25.000000
25%,81900.000000,60.000000,42.000000
50%,96900.000000,60.000000,49.000000
75%,98400.000000,60.000000,57.000000
max,215700.000000,60.000000,102.000000


In [21]:
scale_df1 = StandardScaler().fit_transform(df1[['Amount_Month','Term','Age']])
pd.DataFrame(scale_df1, columns = ['Amount_Month','Term','Age']).describe()

,Amount_Month,Term,Age
count,5.130100e+04,5.130100e+04,4.432900e+04
mean,3.440456e-16,-1.883663e-17,-4.680423e-17
std,1.000010e+00,1.000010e+00,1.000011e+00
min,-2.573947e+00,-3.633629e+00,-2.278282e+00
25%,-7.903086e-01,3.631078e-01,-7.305420e-01
50%,1.898199e-01,3.631078e-01,-9.323703e-02
75%,2.878328e-01,3.631078e-01,6.351114e-01
max,7.952438e+00,3.631078e-01,4.732072e+00


In [22]:
# MinMaxScaler : 최솟값이 0, 최댓값이 1 형태로 데이터를 변환
# 비정형 데이터/ 범주형 데이터 같은 데이터들이 같이 학습이 될 때 주로 사용
scale_df1 = MinMaxScaler().fit_transform(df1[['Amount_Month','Term','Age']])
pd.DataFrame(scale_df1, columns = ['Amount_Month','Term','Age']).describe()

,Amount_Month,Term,Age
count,51301.000000,51301.000000,44329.000000
mean,0.244523,0.909149,0.324988
std,0.095000,0.250207,0.142648
min,0.000000,0.000000,0.000000
25%,0.169444,1.000000,0.220779
50%,0.262556,1.000000,0.311688
75%,0.271867,1.000000,0.415584
max,1.000000,1.000000,1.000000


In [23]:
# RobustScaler : 중앙값 0 IQR 1 형태로 변환
# 비모수적 (이상치, 정규성 X)
scale_df1 = RobustScaler().fit_transform(df1[['Amount_Month','Term','Age']])
pd.DataFrame(scale_df1, columns = ['Amount_Month','Term','Age']).describe()

,Amount_Month,Term,Age
count,51301.000000,51301.000000,44329.000000
mean,-0.176062,-4.360851,0.068273
std,0.927531,12.009915,0.732258
min,-2.563455,-48.000000,-1.600000
25%,-0.909091,0.000000,-0.466667
50%,0.000000,0.000000,0.000000
75%,0.090909,0.000000,0.533333
max,7.200000,0.000000,3.533333


- Label Encoding : 문자를 특정 정수로 변환하여 사용
- One Hot Encoding : 문자를 1/0의 정수를 갖는 Table로 변환하여 사용

In [24]:
pd.get_dummies(df1['Channel'])

,대형마트A,대형마트C,대형마트E,대형마트H,대형마트N,영업방판,자체홈페이지,재계약,전단홍보,전문매장H,전문매장Z,직영계열사A,직영계열사B,직영유통사,홈쇼핑/방송,홈쇼핑/인터넷
0,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False
3,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51296,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
51297,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
51298,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False
51299,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False


# 특성 공학 + 학습

In [25]:
#1 ) 데이터 핸들링
cond1 = (df1['State']=='계약확정')
df1.loc[cond1, 'Target'] = '정상'
df1.loc[~cond1, 'Target'] = '해약'
df1['Target'].value_counts()

Target
정상    50620
해약      681
Name: count, dtype: int64

In [26]:
#2) 목표변수와 설명변수 설정
Y = df1['Target']
X = df1[['Product_Type','Amount_Month', 'Age', 'Gender', 'Credit_Rank', 'Term']]

In [29]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, random_state=1234)

In [39]:
# 파이프라인
from sklearn.pipeline import make_pipeline # 특성 공학 + 학습
# 문자는 문자끼리 / 숫자는 숫자끼리 파이프라인을 병렬로 배치
from sklearn.compose import make_column_transformer
# (특성공학) 1. 결측값 처리
from sklearn.impute import SimpleImputer
# (특성공학) 2. 스케일링/인코딩
from sklearn.preprocessing import StandardScaler, OneHotEncoder
# (학습) 3. 분류 학습 모델
from sklearn.tree import DecisionTreeClassifier

In [40]:
# 숫자/문자데이터를 구분
numeric_list = X.describe().columns
category_list = X.describe(include = 'object').columns

In [41]:
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

In [42]:
#파이프라인 설계 (숫자 데이터: 결측값을 처리(중앙값) -> 스케일링 )
numeric_pipe = make_pipeline(SimpleImputer(strategy = 'median'),
                            StandardScaler())

# 문자 (결측값 (최빈값) -> 인코딩)
category_pipe = make_pipeline(SimpleImputer(strategy='most_frequent'),
                              OneHotEncoder())

In [46]:
# 숫자는 숫자끼리 문자는 문자끼리 처리되는 병렬 파이프를 구성
preprocess_pipe = make_column_transformer((numeric_pipe, numeric_list),
                        (category_pipe, category_list))

In [49]:
# 학습 파이프 구축
model_pipe = make_pipeline(preprocess_pipe, DecisionTreeClassifier())
model_pipe.fit(X_train, Y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('pipeline-1',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('standardscaler',
                                                                   StandardScaler())]),
                                                  Index(['Amount_Month', 'Age', 'Credit_Rank', 'Term'], dtype='object')),
                                                 ('pipeline-2',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehotencoder',
                                                                   OneHotEncoder())]),
                                                  Index(['Product_Type', 'Gender'], dtype='object'))])),
                ('decisiontreeclassifier', DecisionTreeClassifier())])

In [54]:
from sklearn.metrics import precision_score, recall_score
from sklearn.metrics import classification_report

def evaluation_func1(model):
    Y_train_pred = model.predict(X_train)
    Y_test_pred = model.predict(X_test)
    print('학습능력')
    print(classification_report(Y_train, Y_train_pred))
    print('일반화능력')
    print(classification_report(Y_test, Y_test_pred))

In [55]:
# 평가
evaluation_func1(model_pipe)

학습능력
              precision    recall  f1-score   support

          정상       0.99      1.00      0.99     37956
          해약       0.93      0.20      0.33       519

    accuracy                           0.99     38475
   macro avg       0.96      0.60      0.66     38475
weighted avg       0.99      0.99      0.99     38475

일반화능력
              precision    recall  f1-score   support

          정상       0.99      1.00      0.99     12664
          해약       0.02      0.01      0.01       162

    accuracy                           0.98     12826
   macro avg       0.50      0.50      0.50     12826
weighted avg       0.98      0.98      0.98     12826



In [ ]:
pickle.dump(model_pipe, open('model_pipe.sav', 'wb'))

In [57]:
print(X['Product_Type'].unique())
x1 = input('제품 유형을 입력하시오:')
x2 = input('월 랜탈비용을 입력하시오: ')
x3 = input('고객 연령을 입력하시오: ')
x4 = input('고객 성별을 입력하시오(남자/여자):')
x5 = input('고객 신용등급을 입력하시오: ')
x6 = input('계약 기간을 입력하시오: ')

input_data = pd.DataFrame(data=[[x1,x2,x3,x4,x5,x6]], columns=X.columns)

['DES-1' 'DES-3A' 'DES-2' 'DES-R4' 'MMC' 'ERA']
제품 유형을 입력하시오:DES-1
월 랜탈비용을 입력하시오: 100000
고객 연령을 입력하시오: 29
고객 성별을 입력하시오(남자/여자):남자
고객 신용등급을 입력하시오: 1
계약 기간을 입력하시오: 12


In [58]:
model_pipe.predict(input_data)

array(['정상'], dtype=object)

- imputation / Scaling / Encoding : 데이터를 더 적절히 처리하기 위한 특성 공학 기법(실제 학습 모델 성능에는 유의미한 영향을 주지 않음)
- Cross Validation / Hyper Parameter Tuning / imbalanced Data Sampling ... : 학습 모델 성능에 직접적인 영향을 줌

In [59]:
model_pipe

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('pipeline-1',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('standardscaler',
                                                                   StandardScaler())]),
                                                  Index(['Amount_Month', 'Age', 'Credit_Rank', 'Term'], dtype='object')),
                                                 ('pipeline-2',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehotencoder',
                                                                   OneHotEncoder())]),
                                                  Index(['Product_Type', 'Gender'], dtype='object'))])),
                ('decisiontreeclassifier', DecisionTreeClassifier())])

In [61]:
# CV : Cross-Validation
from sklearn.model_selection import GridSearchCV

In [65]:
# 데이터를 5회에 걸쳐 교차검증 실시
grid_model = GridSearchCV(model_pipe, param_grid = {}, cv=5)
# param_grid = {}: HyperParameter Tuning

In [66]:
grid_model.fit(X_train, Y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('columntransformer',
                                        ColumnTransformer(transformers=[('pipeline-1',
                                                                         Pipeline(steps=[('simpleimputer',
                                                                                          SimpleImputer(strategy='median')),
                                                                                         ('standardscaler',
                                                                                          StandardScaler())]),
                                                                         Index(['Amount_Month', 'Age', 'Credit_Rank', 'Term'], dtype='object')),
                                                                        ('pipeline-2',
                                                                         Pipeline(steps=[('simpleimputer',
                                                                                          SimpleImputer(strategy='most_frequent')),
                                                                                         ('onehotencoder',
                                                                                          OneHotEncoder())]),
                                                                         Index(['Product_Type', 'Gender'], dtype='object'))])),
                                       ('decisiontreeclassifier',
                                        DecisionTreeClassifier())]),
             param_grid={})

In [69]:
# 가장 성능이 우수한 모델 선택
best_model = grid_model.best_estimator_
best_model

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('pipeline-1',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('standardscaler',
                                                                   StandardScaler())]),
                                                  Index(['Amount_Month', 'Age', 'Credit_Rank', 'Term'], dtype='object')),
                                                 ('pipeline-2',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehotencoder',
                                                                   OneHotEncoder())]),
                                                  Index(['Product_Type', 'Gender'], dtype='object'))])),
                ('decisiontreeclassifier', DecisionTreeClassifier())])

In [70]:
evaluation_func1(best_model)

학습능력
              precision    recall  f1-score   support

          정상       0.99      1.00      0.99     37956
          해약       0.93      0.20      0.33       519

    accuracy                           0.99     38475
   macro avg       0.96      0.60      0.66     38475
weighted avg       0.99      0.99      0.99     38475

일반화능력
              precision    recall  f1-score   support

          정상       0.99      1.00      0.99     12664
          해약       0.02      0.01      0.01       162

    accuracy                           0.98     12826
   macro avg       0.50      0.50      0.50     12826
weighted avg       0.98      0.98      0.98     12826



In [71]:
pipe1 = make_pipeline(SimpleImputer(strategy='mean'), StandardScaler())
pipe2 = make_pipeline(SimpleImputer(strategy='median'), MinMaxScaler())
pipe3 = make_pipeline(SimpleImputer(strategy='most frequent'), OneHotEncoder())

In [72]:
X.columns

Index(['Product_Type', 'Amount_Month', 'Age', 'Gender', 'Credit_Rank', 'Term'], dtype='object')

In [74]:
multi_pipe = make_column_transformer((pipe1, ['Amount_Month', 'Age']),
                                     (pipe2, ['Credit_Rank', 'Term']),
                                     (pipe3, ['Product_Type', 'Gender']))

In [76]:
model_pipe2 = make_pipeline(multi_pipe, DecisionTreeClassifier())
grid_model = GridSearchCV(model_pipe2, param_grid={}, cv=3)
grid_model.fit(X_train, Y_train)

ValueError: 
All the 3 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
3 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/sinhyelim/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/sinhyelim/anaconda3/lib/python3.11/site-packages/sklearn/pipeline.py", line 401, in fit
    Xt = self._fit(X, y, **fit_params_steps)
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/sinhyelim/anaconda3/lib/python3.11/site-packages/sklearn/pipeline.py", line 359, in _fit
    X, fitted_transformer = fit_transform_one_cached(
                            ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/sinhyelim/anaconda3/lib/python3.11/site-packag

# 개인미션

- TV 홈쇼핑 광고 대행사의 광고 홈쇼핑 방송 송출 데이터

In [3]:
df1 = pd.read_csv('14_Data.csv')

In [79]:
df1.head(2)

,방송Code,채널,소요분,가중분,방송구분,프로그램명,상품ID,상품명,매입과세구분,상품목표취급금액,...,상품취소수량,상품취소금액,ARS금액,매입형태,배송방식,상품소요분,상품가중분,상품방송순번,방송시작시간,방송종료시간
0,1.010036e+09,TV,50.0,52.7,녹화방송,재방_의류,10242138,[비지트인뉴욕콜렉션] 퓨어 캐시미어 롱코트,과세,65645345,...,73,9624136,1000,위탁매입,협력사배송,50.0,52.7,1,2020-01-03T00:10,2020-01-03T01:00
1,1.019126e+09,TV,60.0,38.1,녹화방송,재방_건강식품,10092003,[한삼인]순홍삼진(50ml*30포)*7박스+쇼7_2,과세,23774849,...,16,2547305,1000,위탁매입,직택배,30.0,19.1,1,2020-01-03T01:00,2020-01-03T02:00


-------

### 1. '방송구분'이라는 항목에서 '녹화방송' 에 해당 하는 데이터의 '프로그램명'별 '상품목표주문금액'과 '상품주문금액'의 합과 평균을 계산하시오.

In [84]:
df2 = df1[df1['방송구분']=='녹화방송']
df2.pivot_table(index='프로그램명', values=['상품목표주문금액','상품주문금액'],
               aggfunc = ['sum', 'mean'])

sum                     mean              
           상품목표주문금액     상품주문금액      상품목표주문금액        상품주문금액
프로그램명                                                     
게릴라 프로모션    6386241    8736100  6.386241e+06  8.736100e+06
재방_가전      30944319   41860280  1.031477e+07  1.395343e+07
재방_건강식품    97042110   56468669  2.426053e+07  1.411717e+07
재방_생활용품    31734436   38868330  6.346887e+06  7.773666e+06
재방_속옷      75912426   36617333  3.795621e+07  1.830867e+07
재방_신선농산    86213568   74268755  2.155339e+07  1.856719e+07
재방_의류     471714526  372461954  6.738779e+07  5.320885e+07
재방_이미용     75910061   64767805  1.084429e+07  9.252544e+06
재방_잡화      32719026   32174842  1.635951e+07  1.608742e+07
재방_침구      54728388   47626117  1.368210e+07  1.190653e+07

-----------

### 2. '프로그램명'에서 가장 '상품주문금액'의 합이 큰 3가지를 확인하여, 3가지 항목에 대해 데이터를 추출한 뒤, '상품명'별 '상품주문금액'의 합을 계산하시오.

In [95]:
df3 = df1.sort_values(by='상품주문금액', ascending=False)
# 3가지 잘라내기!
top3program = df3['프로그램명'].head(3)
df4 = df3[df3['프로그램명'].isin(top3program)]

In [96]:
df4.pivot_table(index='상품명', values='상품주문금액',
               aggfunc = 'sum')

,상품주문금액
상품명,
예다함 상조,25351520102
예다함 상조(19.06 사은품변경),11449782736
"예다함 상조(19.08 사은품변경_로봇청소기,에어프라이기,소댕5종)",7466855144
"예다함 상조(19.09 사은품변경_인덕션, 그릴)",14266864780
예다함 상조(4구좌 변경: 공기청정기),1102336452
예다함 상조(4구좌 추가),6418955060


### 강사님풀이

In [7]:
p1 = df1.pivot_table(index='프로그램명',
                values='상품주문금액',aggfunc='sum')
top_list = p1.sort_values(by='상품주문금액', ascending=False).index[:3]

In [10]:
cond1 = df1['프로그램명'].isin(top_list)
df1.loc[cond1].pivot_table(index='상품명', values='상품주문금액', aggfunc='sum')

,상품주문금액
상품명,
모닝애플 조생부사 세척사과 3kg*3박스,83710050
모닝애플 홍로 세척사과 3kg*3박스,260894995
(17년 햅곡)김연도 오색현미 + 옹기가마솥 세트,113245285
(17년 햅곡)김연도 오색현미 22봉,61467733
(17년 햅곡)김연도 웰빙 선물세트,40103888
...,...
황금대추 방울토마토3kg,153834944
황토방 청도 감말랭이 세트(30봉),41407760
황토방 청도 감말랭이 세트(30봉+1봉),106544036


-------------

### 3. '방송시작시간'과 '방송종료시간'의 차이를 계산하여, '방송진행시간'이라는 항목을 선언하고, '방송진행시간'이 1시간 미만인 데이터의 개수를 확인하시오.

In [ ]:
# 시간 형태
# 2020-01-03T00:10
# T는 날짜와 시간을 구분하는 구분자

In [16]:
df1['방송시작시간_dt'] = pd.to_datetime(df1['방송시작시간'])
df1['방송종료시간_dt'] = pd.to_datetime(df1['방송종료시간'])

In [102]:
# (방송종료시간 - 방송시작시간)을 계산하고, 
# 결과를 초로 변환한 후 시간으로 나눔
df1['방송진행시간'] = (df1['방송종료시간_dt'] - df1['방송시작시간_dt']).dt.total_seconds() / 3600

In [106]:
len(df1[df1['방송진행시간'] < 1])

5769

### 강사님풀이

In [11]:
df1['방송진행시간'] = pd.to_datetime(df1['방송종료시간']) - pd.to_datetime(df1['방송시작시간'])
df1['방송진행시간'].describe()

count                        26283
mean     0 days 01:00:28.113229083
std      0 days 00:13:38.395746483
min                0 days 00:10:00
25%                0 days 01:00:00
50%                0 days 01:00:00
75%                0 days 01:05:00
max                1 days 01:05:00
Name: 방송진행시간, dtype: object

In [13]:
cond1 = df1['방송진행시간'] < pd.to_timedelta(1, unit='h')
df1.loc[cond1].shape[0]

5769

----------

### 4. '방송시작시간'을 날짜형식으로 변환하여, '방송연도'/'방송월'/'방송일'/'방송요일'항목을 생성

In [17]:
df1['방송연도'] = df1['방송시작시간_dt'].dt.year
df1['방송월'] = df1['방송시작시간_dt'].dt.month
df1['방송일'] = df1['방송시작시간_dt'].dt.day
df1['방송요일'] = df1['방송시작시간_dt'].dt.day_name()

In [112]:
df1.head(2)

,방송Code,채널,소요분,가중분,방송구분,프로그램명,상품ID,상품명,매입과세구분,상품목표취급금액,...,상품방송순번,방송시작시간,방송종료시간,방송시작시간_dt,방송종료시간_dt,방송진행시간,방송연도,방송월,방송일,방송요일
0,1.010036e+09,TV,50.0,52.7,녹화방송,재방_의류,10242138,[비지트인뉴욕콜렉션] 퓨어 캐시미어 롱코트,과세,65645345,...,1,2020-01-03T00:10,2020-01-03T01:00,2020-01-03 00:10:00,2020-01-03 01:00:00,0.833333,2020,1,3,Friday
1,1.019126e+09,TV,60.0,38.1,녹화방송,재방_건강식품,10092003,[한삼인]순홍삼진(50ml*30포)*7박스+쇼7_2,과세,23774849,...,1,2020-01-03T01:00,2020-01-03T02:00,2020-01-03 01:00:00,2020-01-03 02:00:00,1.000000,2020,1,3,Friday


------------

### 5. '방송요일'항목에서 주말과 주중을 구분하여, '주말'과 '주중'의 '상품주문수량'과 '상품주문금액'의 평균의 차이를 계산

In [126]:
cond1 = (df1['방송요일'] == 'Saturday')
cond2 = (df1['방송요일'] == 'Sunday')

weekend_df = df1[cond1|cond2]
weekday_df = df1[~(cond1|cond2)]

In [130]:
quantity_diff = abs(weekend_df['상품주문수량'].mean() - weekday_df['상품주문수량'].mean())
amount_diff = abs(weekend_df['상품주문금액'].mean() - weekday_df['상품주문금액'].mean())

print(f'주말과 주중의 상품주문수량의 평균의 차이는 {quantity_diff}입니다.')
print(f'주말과 주중의 상품주문금액의 평균의 차이는 {amount_diff}입니다.')

주말과 주중의 상품주문수량의 평균의 차이는 16.6918846481193입니다.
주말과 주중의 상품주문금액의 평균의 차이는 11246180.77671355입니다.


### 강사님풀이

In [18]:
cond1 = df1['방송요일'].isin(['Saturday','Sunday'])

df1.loc[cond1, '주말구분'] = '주말'
df1.loc[~cond1, '주말구분'] = '주중'
df1['주말구분'].value_counts()

주말구분
주중    18797
주말     7486
Name: count, dtype: int64

In [19]:
df1.loc[cond1]['상품주문수량'].mean() - df1.loc[~cond1]['상품주문수량'].mean()

16.6918846481193

In [21]:
df1.loc[cond1]['상품주문금액'].mean() - df1.loc[~cond1]['상품주문금액'].mean()

-11246180.77671355

----------

 ### 6. '배송방식'과 '매입형태'간의 연관성이 있는지 확인하고자 한다.
    - Contingency Table을 계산하고, 두 항목간 독립성 검정을 수행

In [132]:
contingency_table = pd.crosstab(df1['배송방식'], df1['매입형태'])
contingency_table

매입형태,위탁매입,직매입
배송방식,,
당사배송,1,1019
직택배,996,0
협력사배송,24267,0


In [134]:
# 독립성 검정 (Chi^2 Test)
# 귀무가설 : 두 데이터는 서로 독립이다.
# 대립가설 : 두 데이터는 서로 독립이 아니다.
stats.chi2_contingency(contingency_table)
# P.value < 0.05 / 대립가설 참 / 두 데이터는 서로 독립이 아니다.

Chi2ContingencyResult(statistic=26256.193038804344, pvalue=0.0, dof=2, expected_freq=array([[  980.45428604,    39.54571396],
       [  957.38477343,    38.61522657],
       [23326.16094053,   940.83905947]]))

------------

### 7. '방송구분'에서 '생방송'과 '녹화방송'의 매출을 비교하고자 한다. '생방송'과'녹화방송'의 '상품주문금액'의 평균의 차이가 있는지 검정
    - 가설 수립 및 결론 작성

In [137]:
df_live = df1[df1['방송구분']=='생방송']
df_rec = df1[df1['방송구분']=='녹화방송']

In [138]:
# 3-1. 각 집단 별 '상품주문금액' 값의 정규성 검정부터 실시해야 함

# 귀무가설 : 해당 연속형 데이터의 분포는 정규분포를 따른다.
# 대립가설 : 해당 연속형 데이터의 분포는 정규분포를 따르지 않는다.
print(stats.normaltest(df_live['상품주문금액']))
#p.value < 0.05 / 정규분포를 따르지 않음
print(stats.normaltest(df_rec['상품주문금액']))
#p.value < 0.05 /정규분포를 따르지 않음

NormaltestResult(statistic=17929.951096292796, pvalue=0.0)
NormaltestResult(statistic=37.59192300234786, pvalue=6.870963444477694e-09)


In [143]:
# 3-2. (정규분포를 따르지 않는/비모수)두 집단의 중앙값을 비교

# 귀무가설 : 두 집단의 해당 연속형 자료의 중앙값은 서로 차이가 없다.
# 대립가설 : 두 집단의 해당 연속형 자료의 중앙값은 서로 차이가 있다.
stats.ranksums(df_live['상품주문금액'],df_rec['상품주문금액'])
# P.value < 0.05/ 대립가설 참 / 두 집단의 해당 연속형 자료의 중앙값은 서로 차이가 있다.

RanksumsResult(statistic=5.897121686301845, pvalue=3.698970792035826e-09)

--------

### 8. '상품목표주문금액'에서 '상품주문금액'의 차이를 계산해, 목표를 달성하지 못한 방송에 대해서는 0 / 목표를 달성한 방송은 1값으로 '목표달성여부' 변수를 생성

In [151]:
def func8(row):
    if row['상품주문금액'] >= row['상품목표주문금액']:
        return 1
    else:
        return 0

# apply 메서드를 사용하여 '목표달성여부' 변수를 생성합니다.
# 변수를 생성하려는 거기 때문에, 행에 함수를 적용해야함
# 따라서 axis=1 이라는 옵션 설정 (디폴트는 axis=0이며 열에 설정)
df1['목표달성여부'] = df1.apply(func8, axis=1)

In [152]:
df1

,방송Code,채널,소요분,가중분,방송구분,프로그램명,상품ID,상품명,매입과세구분,상품목표취급금액,...,방송시작시간,방송종료시간,방송시작시간_dt,방송종료시간_dt,방송진행시간,방송연도,방송월,방송일,방송요일,목표달성여부
0,1.010036e+09,TV,50.0,52.7,녹화방송,재방_의류,10242138,[비지트인뉴욕콜렉션] 퓨어 캐시미어 롱코트,과세,65645345,...,2020-01-03T00:10,2020-01-03T01:00,2020-01-03 00:10:00,2020-01-03 01:00:00,0.833333,2020,1,3,Friday,0
1,1.019126e+09,TV,60.0,38.1,녹화방송,재방_건강식품,10092003,[한삼인]순홍삼진(50ml*30포)*7박스+쇼7_2,과세,23774849,...,2020-01-03T01:00,2020-01-03T02:00,2020-01-03 01:00:00,2020-01-03 02:00:00,1.000000,2020,1,3,Friday,1
2,1.019226e+09,TV,60.0,38.1,녹화방송,재방_건강식품,10092743,[단품_한삼인] 순홍삼진(50ml*30포)*1박스,과세,23774849,...,2020-01-03T01:00,2020-01-03T02:00,2020-01-03 01:00:00,2020-01-03 02:00:00,1.000000,2020,1,3,Friday,0
3,1.040337e+09,TV,60.0,19.7,재방송,재방_가공농산,10295865,이상용의 우리밀 크레마롤,과세,24568911,...,2020-01-03T02:00,2020-01-03T03:00,2020-01-03 02:00:00,2020-01-03 03:00:00,1.000000,2020,1,3,Friday,0
4,1.021427e+09,TV,60.0,16.1,재방송,재방_신선수산,10113666,국내산 자숙 문어슬라이스 120g*9팩,면세,20131383,...,2020-01-03T03:00,2020-01-03T04:00,2020-01-03 03:00:00,2020-01-03 04:00:00,1.000000,2020,1,3,Friday,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26278,1.046263e+13,TV,60.0,94.3,생방송,가전,11179512,■연말특가■제우스커브드 UHD TV 65형,과세,92367433,...,2021-11-13T21:45,2021-11-13T22:45,2021-11-13 21:45:00,2021-11-13 22:45:00,1.000000,2021,11,13,Saturday,0
26279,1.046263e+13,TV,60.0,94.3,생방송,가전,11179514,2019 제우스 UHD TV 75형,과세,33698544,...,2021-11-13T21:45,2021-11-13T22:45,2021-11-13 21:45:00,2021-11-13 22:45:00,1.000000,2021,11,13,Saturday,1
26280,1.046263e+13,TV,60.0,94.3,생방송,가전,11179504,■연말특가■제우스커브드 UHD TV 55형,과세,17202608,...,2021-11-13T21:45,2021-11-13T22:45,2021-11-13 21:45:00,2021-11-13 22:45:00,1.000000,2021,11,13,Saturday,1
26281,1.020263e+13,TV,60.0,91.6,재방송,재방_건강식품,11152474,[래오이경제] 흑염소진액 70ml x 120포,과세,154250864,...,2021-11-13T22:45,2021-11-13T23:45,2021-11-13 22:45:00,2021-11-13 23:45:00,1.000000,2021,11,13,Saturday,0


----------

### 9. '목표달성여부'를 분류하는 모델을 만들고자 한다.
    - X : 소요분 / 방송구분 / 판매단가 / ARS 금액 /수수료율 / '방송요일' / '방송월'
    - Y : '목표달성여부'
    - 학습 : 검증 = 8 : 2
    - 특성 공학 기법 (결측처리(평균,최빈값) + 스케일링&인코딩)
    - 알고리즘 (Decision Tree 알고리즘 / 하이퍼파라미터 튜닝 X)
    - 평가

In [154]:
Y = df1['목표달성여부']
X = df1[['소요분', '방송구분', '판매단가', 'ARS금액', '수수료율', '방송요일', '방송월']]

In [22]:
### 필요 라이브러리 불러오기 ###
#학습데이터와 검증 데이터 분할
from sklearn.model_selection import train_test_split
# 학습과 특성공학이 같이 수행되는 파이프라인 구축
from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_transformer
#특성공학 기법
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
#알고리즘
from sklearn.tree import DecisionTreeClassifier
#교차검증 + 하이퍼 파라미터 튜닝 기법
from sklearn.model_selection import GridSearchCV
# 평가
from sklearn.metrics import classification_report

In [155]:
from sklearn.model_selection import train_test_split

In [156]:
# 학습 및 검증 데이터셋 나누기 (8:2)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y,
                                                    test_size=0.2,
                                                    random_state=1234)

In [159]:
# 파이프라인 생성

In [161]:
# 숫자 / 문자 데이터를 구분
numeric_list = X.describe().columns
category_list = X.describe(include = 'object').columns

In [162]:
#파이프라인 설계 (숫자 데이터: 결측값을 처리(평균) -> 스케일링 )
numeric_pipe = make_pipeline(SimpleImputer(strategy = 'mean'),
                            StandardScaler())

# 문자 (결측값 (최빈값) -> 인코딩)
category_pipe = make_pipeline(SimpleImputer(strategy='most_frequent'),
                              OneHotEncoder())

In [163]:
# 숫자는 숫자끼리 문자는 문자끼리 처리되는 병렬 파이프를 구성
preprocess_pipe = make_column_transformer((numeric_pipe, numeric_list),
                        (category_pipe, category_list))

In [165]:
# 학습 파이프 구축
model_pipe2 = make_pipeline(preprocess_pipe, DecisionTreeClassifier())
model_pipe2.fit(X_train, Y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('pipeline-1',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer()),
                                                                  ('standardscaler',
                                                                   StandardScaler())]),
                                                  Index(['소요분', '판매단가', 'ARS금액', '수수료율', '방송월'], dtype='object')),
                                                 ('pipeline-2',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehotencoder',
                                                                   OneHotEncoder())]),
                                                  Index(['방송구분', '방송요일'], dtype='object'))])),
                ('decisiontreeclassifier', DecisionTreeClassifier())])

In [166]:
evaluation_func1(model_pipe2)

학습능력
              precision    recall  f1-score   support

           0       0.95      1.00      0.97     13042
           1       0.99      0.91      0.95      7984

    accuracy                           0.96     21026
   macro avg       0.97      0.95      0.96     21026
weighted avg       0.96      0.96      0.96     21026

일반화능력
              precision    recall  f1-score   support

           0       0.70      0.70      0.70      3291
           1       0.51      0.51      0.51      1966

    accuracy                           0.63      5257
   macro avg       0.61      0.61      0.61      5257
weighted avg       0.63      0.63      0.63      5257

